In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
from tensorflow.python.keras import layers, Sequential,losses, metrics, optimizers, callbacks, models
from tensorflow.python.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.applications import vgg16, resnet
from tensorflow.keras.optimizers import SGD, RMSprop, Adam

In [ ]:
image_height = 48
image_width = 48
emotions_count = 8
emotion_labels = ['neutral', 'happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear', 'contempt']

In [ ]:
# dataset_index:
#     0: original samples
#     1: original samples + flip
#     2: original samples + flip ± rotation 10°
#     3: original samples + flip ± rotation 20°
#     4: original samples + flip ± rotation 30°
#     5: original samples + flip ± rotation 40°
# resize:
#     0: pixel value 0~255
#     1: pixel value -1~1
#     2: pixel value 0~1
def load_data(dataset_index, resize):
    if dataset_index == 0:
        image_path = "./dataset/images.npy"
        emotion_path = "./dataset/emotions_multi.npy"
        training_sample_times = 1
    if dataset_index == 1:
        image_path = "./dataset/FERPlus/images.npy"
        emotion_path = "./dataset2/emotions_multi.npy"
        training_sample_times = 2
    if dataset_index == 2:
        image_path = "./dataset_f+r10/images.npy"
        emotion_path = "./dataset_f+r10/emotions_multi.npy"
        training_sample_times = 4
    if dataset_index == 3:
        image_path = "./dataset_f+r20/images.npy"
        emotion_path = "./dataset_f+r20/emotions_multi.npy"
        training_sample_times = 4
    if dataset_index == 4:
        image_path = "./dataset_f+r30/images.npy"
        emotion_path = "./dataset_f+r30/emotions_multi.npy"
        training_sample_times = 4
    if dataset_index == 5:
        image_path = "./dataset_f+r40/images.npy"
        emotion_path = "./dataset_f+r40/emotions_multi.npy"
        training_sample_times = 4
    images = np.load(image_path)
    images = tf.convert_to_tensor(images)
    if resize == 0:
        pass
    if resize == 1:
        images = layers.Rescaling(1./127.5, offset= -1)(images)
    if resize == 2:
        images = layers.Rescaling(1./255, offset= 0)(images)
    images = tf.image.grayscale_to_rgb(images)
    emotions = np.load(emotion_path)
    emotions = tf.convert_to_tensor(emotions)
    
    training_samples = 28317 * training_sample_times
    validation_samples = 3541 * training_sample_times
    training_size = training_samples + validation_samples
    
    training_images = images[:training_size]
    test_images = images[training_size:]
    training_emotions = emotions[:training_size]
    test_emotions = emotions[training_size:]
    return training_images, training_emotions, test_images, test_emotions

training_images, training_emotions, test_images, test_emotions = load_data(1, 1)

In [ ]:
tf.config.run_functions_eagerly(True)
def model_acc(y_true, y_pred):
    size = y_true.shape[0]
    acc = 0
    for i in range(size):
        true = y_true[i]
        pred = y_pred[i]           
        index_max = tf.argmax(pred).numpy()
        if true[index_max].numpy()==tf.reduce_max(true).numpy():
            acc += 1
    return acc/size

In [ ]:
def train(model, optimizer, loss, num_epochs, batch_size):
    model.compile(optimizer=optimizer, 
                  loss=loss, 
                  metrics = [model_acc])
    return model.fit(x=training_images,
                     y=training_emotions,
                     batch_size=batch_size,
                     epochs=num_epochs,
                     validation_data=(test_images, test_emotions))

In [ ]:
def create_model():
    base_model = vgg16.VGG16(include_top=False, 
                             weights="imagenet", 
                             input_shape=(48,48,3))
    base_model.trainable=True
    return Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(4096, activation='relu'),
        layers.Dense(4096, activation='relu'),
        layers.Dense(emotions_count, activation='softmax'),
    ])

In [ ]:
if not os.path.isdir('./new-results/'):
    os.mkdir('./new-results/')

num_epochs = 40
learning_rate = 1e-4
loss = losses.MeanSquaredError()
batch_size = 32
val_accs = []
for opt in range(3):
    model = create_model()
    if opt == 0:
        optimizer = Adam(learning_rate=learning_rate)
        history_save_path = './new-results/Adam.txt'
    if opt == 1:
        optimizer = SGD(learning_rate=learning_rate)
        history_save_path = './new-results/SGD.txt'
    if opt == 2:
        optimizer = RMSprop(learning_rate=learning_rate)
        history_save_path = './new-results/RMSprop.txt'
    history = train(model, optimizer, loss, num_epochs, batch_size)
    val_accs.append(history.history['val_model_acc'])
    with open(history_save_path, 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

In [ ]:
# model comparison
epochs = range(1, 1+num_epochs)
fig = plt.figure()
for opt in range(3):
    val_acc = val_accs[opt]
    if opt == 0:
        plot_label = 'Adam'
    if opt == 1:
        plot_label = 'SGD'
    if opt == 2:
        plot_label = 'RMSprop'
    plt.plot(epochs, val_acc, label = plot_label)
plt.legend()
plt.show()

# save plot
fig.savefig('./new-results/optimizers.png')